In [173]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("parsed_training.csv")

In [3]:
df.head()

,file_name,instance_number,part_level,is_occluded,class_name,raw_name,attr_list
0,ADE20K_2016_07_26/images/training/v/valley/ADE...,1,0,0,sky,sky,[]
1,ADE20K_2016_07_26/images/training/v/valley/ADE...,2,0,0,field,field,[]
2,ADE20K_2016_07_26/images/training/v/valley/ADE...,3,0,0,river,river water,[]
3,ADE20K_2016_07_26/images/training/v/valley/ADE...,4,0,0,tree,trees,[]
4,ADE20K_2016_07_26/images/training/v/valley/ADE...,5,0,0,field,field,['grass']


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538372 entries, 0 to 538371
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   file_name        538372 non-null  object
 1   instance_number  538372 non-null  int64 
 2   part_level       538372 non-null  int64 
 3   is_occluded      538372 non-null  int64 
 4   class_name       538372 non-null  object
 5   raw_name         538372 non-null  object
 6   attr_list        538372 non-null  object
dtypes: int64(3), object(4)
memory usage: 28.8+ MB


In [5]:
df.raw_name.describe()

count     538372
unique      2943
top       window
freq       42139
Name: raw_name, dtype: object

In [6]:
class_images = df.groupby(["raw_name", "file_name"]).size().groupby(level=0).size()

In [7]:
class_images[class_images >= 100]

raw_name
aerial               122
air conditioning     313
airplane             133
aperture             175
apron                363
                    ... 
wheel               1076
window              8331
windows              211
windshield           544
work surface         696
Length: 301, dtype: int64

In [8]:
df[df.file_name == "ADE20K_2016_07_26/images/training/v/vegetable_garden/ADE_train_00019477_atr.txt"]

,file_name,instance_number,part_level,is_occluded,class_name,raw_name,attr_list
199,ADE20K_2016_07_26/images/training/v/vegetable_...,1,0,0,sky,sky,[]
200,ADE20K_2016_07_26/images/training/v/vegetable_...,2,0,0,"road, route",road,[]
201,ADE20K_2016_07_26/images/training/v/vegetable_...,3,0,0,"building, edifice",building,[]
202,ADE20K_2016_07_26/images/training/v/vegetable_...,4,0,0,"building, edifice",building,[]
203,ADE20K_2016_07_26/images/training/v/vegetable_...,5,0,0,"building, edifice",building,[]
204,ADE20K_2016_07_26/images/training/v/vegetable_...,6,0,0,grass,grass,[]
205,ADE20K_2016_07_26/images/training/v/vegetable_...,7,0,0,grass,grass,[]
206,ADE20K_2016_07_26/images/training/v/vegetable_...,8,0,0,"building, edifice",building,[]
207,ADE20K_2016_07_26/images/training/v/vegetable_...,9,0,1,"car, auto, automobile, machine, motorcar",car,[]
208,ADE20K_2016_07_26/images/training/v/vegetable_...,10,0,0,"plant, flora, plant life",plant,[]


In [12]:
df.raw_name.nunique(), df.class_name.nunique()

(2943, 2904)

In [16]:
classes = df.groupby("class_name").size()
classes[classes > 100].shape

(369,)

In [151]:
fname

,0,1,2,3,4,5,6
0,ADE20K_2016_07_26,images,training,v,valley,ADE_train_00019465_atr.txt,None
1,ADE20K_2016_07_26,images,training,v,valley,ADE_train_00019465_atr.txt,None
2,ADE20K_2016_07_26,images,training,v,valley,ADE_train_00019465_atr.txt,None
3,ADE20K_2016_07_26,images,training,v,valley,ADE_train_00019465_atr.txt,None
4,ADE20K_2016_07_26,images,training,v,valley,ADE_train_00019465_atr.txt,None
...,...,...,...,...,...,...,...
538367,ADE20K_2016_07_26,images,training,s,subway_interior,ADE_train_00018928_atr.txt,None
538368,ADE20K_2016_07_26,images,training,s,subway_interior,ADE_train_00018928_atr.txt,None
538369,ADE20K_2016_07_26,images,training,s,subway_interior,ADE_train_00018928_atr.txt,None
538370,ADE20K_2016_07_26,images,training,s,subway_interior,ADE_train_00018928_atr.txt,None


In [138]:
fname = pd.DataFrame(df.file_name.str.split("/").values.tolist())
df["image_id"] = fname[6].combine_first(fname[5]).combine_first(fname[4]).str.extract("(\d+)")

In [139]:
df.sort_values("image_id")

,file_name,instance_number,part_level,is_occluded,class_name,raw_name,attr_list,scene,image_id
9433,ADE20K_2016_07_26/images/training/a/airport_te...,65,0,1,"person, individual, someone, somebody, mortal,...",person,[],airport_terminal,00000001
9441,ADE20K_2016_07_26/images/training/a/airport_te...,73,0,0,"bag, traveling bag, travelling bag, grip, suit...",suitcase,[],airport_terminal,00000001
9440,ADE20K_2016_07_26/images/training/a/airport_te...,72,0,1,"person, individual, someone, somebody, mortal,...",person,[],airport_terminal,00000001
9439,ADE20K_2016_07_26/images/training/a/airport_te...,71,0,0,"person, individual, someone, somebody, mortal,...",person,[],airport_terminal,00000001
9438,ADE20K_2016_07_26/images/training/a/airport_te...,70,0,1,"person, individual, someone, somebody, mortal,...",person,[],airport_terminal,00000001
...,...,...,...,...,...,...,...,...,...
92136,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,5,0,0,"person, individual, someone, somebody, mortal,...",person,['sitting'],zoo,00020210
92135,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,4,0,0,tree,trees,[],zoo,00020210
92134,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,3,0,0,"building, edifice",building,[],zoo,00020210
92147,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,16,0,0,"animal, animate being, beast, brute, creature,...",animal,['giraffe'],zoo,00020210


In [160]:
def get_scene(row):
    scene = "/".join([row[4] or "", row[5] or "", row[6] or ""])
    return scene[:scene.find("ADE")].rstrip("/") or row[3]

scenes = fname.apply(get_scene, axis=1)

In [162]:
df["scene"] = scenes.values

In [166]:
df.sort_values(["image_id", "instance_number"])

,file_name,instance_number,part_level,is_occluded,class_name,raw_name,attr_list,scene,image_id
9369,ADE20K_2016_07_26/images/training/a/airport_te...,1,0,0,wall,wall,['glass'],airport_terminal,00000001
9370,ADE20K_2016_07_26/images/training/a/airport_te...,2,0,0,wall,wall,['glass'],airport_terminal,00000001
9371,ADE20K_2016_07_26/images/training/a/airport_te...,3,0,0,wall,wall,['glass'],airport_terminal,00000001
9372,ADE20K_2016_07_26/images/training/a/airport_te...,4,0,0,wall,wall,[],airport_terminal,00000001
9373,ADE20K_2016_07_26/images/training/a/airport_te...,5,0,0,wall,wall,[],airport_terminal,00000001
...,...,...,...,...,...,...,...,...,...
92157,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,26,0,1,"ashcan, trash can, garbage can, wastebin, ash ...",trash can,[],zoo,00020210
92158,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,27,0,0,"fence, fencing",fence,[],zoo,00020210
92159,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,28,0,0,"fence, fencing",fence,[],zoo,00020210
92160,ADE20K_2016_07_26/images/training/z/zoo/ADE_tr...,29,0,1,"animal, animate being, beast, brute, creature,...",animal,['zebra'],zoo,00020210


In [167]:
"00000001"

'00000001'

In [37]:
import scipy.io
mat = scipy.io.loadmat('ADE20K_2016_07_26/index_ade20k.mat')

In [38]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Jul 29 08:20:25 2016',
 '__version__': '1.0',
 '__globals__': [],
 'index': array([[(array([[array(['ADE_train_00000001.jpg'], dtype='<U22'),
         array(['ADE_train_00000002.jpg'], dtype='<U22'),
         array(['ADE_train_00000003.jpg'], dtype='<U22'), ...,
         array(['ADE_val_00001998.jpg'], dtype='<U20'),
         array(['ADE_val_00001999.jpg'], dtype='<U20'),
         array(['ADE_val_00002000.jpg'], dtype='<U20')]], dtype=object), array([[array(['ADE20K_2016_07_26/images/training/a/airport_terminal'],
       dtype='<U52'),
         array(['ADE20K_2016_07_26/images/training/a/airport_terminal'],
       dtype='<U52'),
         array(['ADE20K_2016_07_26/images/training/a/art_gallery'], dtype='<U47'),
         ...,
         array(['ADE20K_2016_07_26/images/validation/w/waterfall/block'],
       dtype='<U53'),
         array(['ADE20K_2016_07_26/images/validation/w/waterfall/fan'],
       dtype='<U51'),
   

In [99]:
mat["index"].dtype

dtype('O')

In [234]:
classes = []
for x in mat["index"][0][0][6][0]:
    classes.append(x[0])

In [329]:
import imageio
mask = imageio.imread("ADE20K_2016_07_26/images/training/a/airport_terminal/ADE_train_00000001_seg.png")
mask = np.array(mask, np.uint16)

In [330]:
np.unique(mask[:, :, 2])

array([  0,   3,   5,   8,  10,  13,  15,  18,  20,  23,  25,  28,  30,
        33,  35,  38,  40,  43,  45,  48,  50,  53,  55,  58,  60,  63,
        65,  68,  70,  73,  75,  78,  80,  83,  85,  88,  90,  93,  95,
        98, 100, 103, 105, 108, 110, 113, 115, 118, 120, 123, 125, 128,
       130, 133, 135, 138, 140, 143, 145, 148, 150, 153, 155, 158, 160,
       163, 165, 168, 170, 173, 175, 178, 180, 183, 185, 188, 190, 193,
       195, 198, 200, 203, 205, 208, 210, 213, 215, 218, 220, 223, 225,
       228, 230, 233, 235, 238, 240, 243, 245, 248, 250, 253, 255],
      dtype=uint16)

In [337]:
label_mask = (mask[:, :, 0] // 10) * 256 + mask[:, :, 1]
labeled = np.unique(label_mask)
labeled = labeled[labeled!=0] -1
sorted(np.array(classes)[labeled])

['ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin',
 'bag, traveling bag, travelling bag, grip, suitcase',
 'ceiling',
 'column, pillar',
 'elevator, lift',
 'escalator, moving staircase, moving stairway',
 'fence, fencing',
 'first floor',
 'flag',
 'floor, flooring',
 'fountain',
 'person, individual, someone, somebody, mortal, soul',
 'plant, flora, plant life',
 'pot, flowerpot',
 'seat',
 'signboard, sign',
 'streetlight, street lamp',
 'tree',
 'wall']

In [336]:
sorted(df[df.image_id=="00000001"].class_name.unique())

['ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin',
 'bag, traveling bag, travelling bag, grip, suitcase',
 'ceiling',
 'column, pillar',
 'elevator, lift',
 'escalator, moving staircase, moving stairway',
 'fence, fencing',
 'first floor',
 'flag',
 'floor, flooring',
 'fountain',
 'person, individual, someone, somebody, mortal, soul',
 'plant, flora, plant life',
 'pot, flowerpot',
 'seat',
 'signboard, sign',
 'streetlight, street lamp',
 'tree',
 'wall']

array([  -1,   63,   96,  446,  580,  852,  867,  906,  946,  953,  975,
       1022, 1830, 1909, 1980, 2271, 2379, 2615, 2854, 2977], dtype=int16)